In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import random
import string
import hashlib
import json

#Loading data
columns_name = ['id', 'date', 'latitude', 'longitude' ]
file_name = 'DAZC_481'

df = pd.read_csv('database', names = columns_name, sep = '\t')
df['date'] = df['date'].str.slice(stop = 16)
df['date'] = pd.to_datetime(df['date'], format = "%Y-%m-%d %H:%M")
df['week'] = df['date'].dt.isocalendar().week


MemoryError: Unable to allocate 264. MiB for an array with shape (1, 34551849) and data type object

In [ ]:
anon = pd.read_csv('./files/'+file_name, names = columns_name, sep = '\t')
anon = anon.loc[anon.id != 'DEL']
anon['date'] = anon['date'].str.slice(stop = 16)
anon['date'] = pd.to_datetime(anon['date'], format = "%Y-%m-%d %H:%M")
anon['week'] = anon['date'].dt.isocalendar().week
anon.latitude = anon.latitude.astype(float)
anon.longitude = anon.longitude.astype(float)
#print(anon.head())

In [ ]:
#List of id origin, id anonym, group of id-week

id_week_anon = anon.groupby('id')['week'].unique()
id_week_ori = df.groupby('id')['week'].unique()

list_id_anon = anon["id"].unique()
list_id_ori = df["id"].unique()
#print(np.sort(list_id_anon), len(list_id_anon))

In [4]:
#Group id--week--moyen_latitude--moyen_longitude--mean_date--count_line--mean_hour

#Table origin
df['hour'] = df['date'].dt.hour
mean_hour = df.groupby(['id','week'])['hour'].mean()

mean_lat_ori = df.groupby(['id','week'])['latitude'].mean()
mean_long_ori = df.groupby(['id','week'])['longitude'].mean()

mean_date_ori = df.groupby(['id','week'])['date'].mean()
count_ligne_ori = df.groupby(['id','week'])['date'].count()
mean_hour = df.groupby(['id','week'])['hour'].mean()

ori_mean = df[['id','week']].drop_duplicates().reset_index(drop=True)
ori_mean['mean_latitude'] = ori_mean.apply(lambda x: mean_lat_ori[x.id][x.week], axis=1)
ori_mean['mean_longitude'] = ori_mean.apply(lambda x: mean_long_ori[x.id][x.week], axis=1)
ori_mean['mean_date'] = ori_mean.apply(lambda x: mean_date_ori[x.id][x.week], axis=1)
ori_mean['count_line'] = ori_mean.apply(lambda x: count_ligne_ori[x.id][x.week], axis=1)
ori_mean['mean_hour'] = ori_mean.apply(lambda x: mean_hour[x.id][x.week], axis=1)

#Table anonym
anon['hour'] = anon['date'].dt.hour
mean_lat_anon = anon.groupby(['id','week'])['latitude'].mean()
mean_long_anon = anon.groupby(['id','week'])['longitude'].mean()
mean_date_anon = anon.groupby(['id','week'])['date'].mean()
count_ligne_anon = anon.groupby(['id','week'])['date'].count()
mean_hour_anon = anon.groupby(['id','week'])['hour'].mean()

anon_mean = anon[['id','week']].drop_duplicates().reset_index(drop=True)
anon_mean['mean_latitude'] = anon_mean.apply(lambda x: mean_lat_anon[x.id][x.week] , axis=1)
anon_mean['mean_longitude'] = anon_mean.apply(lambda x: mean_long_anon[x.id][x.week], axis=1)
anon_mean['mean_date'] = anon_mean.apply(lambda x: mean_date_anon[x.id][x.week], axis=1)
anon_mean['count_line'] = anon_mean.apply(lambda x: count_ligne_anon[x.id][x.week], axis=1)
anon_mean['mean_hour'] = anon_mean.apply(lambda x: mean_hour_anon[x.id][x.week], axis=1)

In [7]:
ori_mean['new_id'] = ori_mean['id']
scale = anon.shape[0] / df.shape[0]
for i in list_id_ori:
    for w in id_week_ori[i]:
        list_distance = list() 
        mean_lat = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_latitude'].to_list()[0]
        mean_long = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_longitude'].to_list()[0]
        mean_date = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_date'].to_list()[0]
        number_line = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'count_line'].to_list()[0]
        mean_hour = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_hour'].to_list()[0]
        
        group_week = anon_mean.loc[anon_mean.week ==w,['id','week','mean_latitude','mean_longitude','count_line','mean_date','mean_hour']].reset_index(drop=True)
        for j in range(len(group_week)):
            distance = np.power((group_week.mean_latitude[j] - mean_lat)**2 + (group_week.mean_longitude[j] - mean_long)**2, 1/2)
            
            delta_ratio = np.power(abs((group_week.count_line[j]/anon.shape[0] - number_line/df.shape[0])),4)
            delta_time = np.power(abs((group_week.mean_date[j] - mean_date).total_seconds() / 3600),1)
            
            
            delta_hour = np.power(abs((group_week.mean_hour[j] - mean_hour)) ,1/2)
            
            list_distance.append(distance*delta_count*delta_hour) #*delta_count*delta_time*distance*delta_hour
            
        index = list_distance.index(min(list_distance))
        l = id_week_anon[group_week.id[index]]# list of week have id anon_mean.id[index] 
        while ((w not in l)):
            list_distance[index] = 1000000000
            index = list_distance.index(min(list_distance).all())
            l = id_week_anon[group_week.id[index]]
        if (w in l):
            ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'new_id'] = group_week.id[index]
        
#print(ori_mean.head())
print(ori_mean.new_id.nunique())

result = {}
count2 = 0
for i in list_id_ori:
    result[str(i)] = {}
    for w in id_week_ori[i]:
        result[str(i)]['2015'+'-'+str(w)] = [str(ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'new_id'][count2])]
        count2 +=1
with open('attack_'+file_name, 'w') as fp:
    json.dump(result, fp, indent=1)


239


In [60]:
#df['day'] = df['date'].dt.day
df['hour'] = df['date'].dt.hour
ori_count_day = df[['id','week','day']].drop_duplicates().reset_index(drop=True)
count_ori = df.groupby(['id','week','day'])['day'].count()
ori_count_day['count'] = ori_count_day.apply(lambda x: count_ori[x.id][x.week][x.day], axis=1)

anon['day'] = anon['date'].dt.day
anon['hour'] = anon['date'].dt.hour
anon_count_day = anon[['id','week','day']].drop_duplicates().reset_index(drop=True)
count_anon = anon.groupby(['id','week','day'])['day'].count()
anon_count_day['count'] = anon_count_day.apply(lambda x: count_anon[x.id][x.week][x.day], axis=1)

ori_count_day = ori_count_day.sort_values('count')
anon_count_day = anon_count_day.sort_values('count')
print(ori_count_day)
print(anon_count_day)

      id  week  day  count
1005  41    11   10      1
34     1    15    7      1
71     2    10    5      1
648   13    11   14      1
692   13    20   11      2
...   ..   ...  ...    ...
2209   4    19    8  60620
1256  62    10    4  60826
1304  62    19    4  61414
1265  62    11   13  61813
781   28    12   22  61846

[2223 rows x 4 columns]
              id  week  day  count
2139  ?SBHZCa~#5    20   13      1
919    yko6M2b._    15   11      1
909    <xOs8C>tb    12   22      1
2142  ?SBHZCa~#5    20   12      1
2141   vByHr)d3!    12   22      1
...          ...   ...  ...    ...
2228  @-v/S>{FO7    10    7  51507
1365   "*=SlkE$+    10    7  52615
2146  /BLyN)$Mh.    12   17  53966
1010  -j%ZAGf)62    19    9  55462
355    Y<R;XoKD`    10    7  56752

[2948 rows x 4 columns]


In [394]:
anon_day_count = anon.groupby(['id','week','day'])['day'].count()

ori_mean = df[['id','week']].drop_duplicates().reset_index(drop=True)

id          week  day
"'1SyzA)c5  14    1      20866
                  2      19920
                  3      23598
                  4      24844
                  5      23056
                         ...  
zuE]d~pD4   10    4       7246
                  5       8460
                  6      15832
                  7      12246
                  8       8469
Name: day, Length: 2948, dtype: int64

In [73]:

list_ori = list()
list_ano_id = list()
one_time = list()
file_compare = 'attack_DAZC_481_no_distance'
with open(file_compare) as o:
    ori_file = json.load(o)
    for _id in ori_file:
        for week in ori_file[_id]:
            list_ori.append(str(_id)+'-'+week[5:])
            list_ano_id.append(ori_file[_id][week][0])
#print(list_ori, len(list_ori))
list_diff = list()
with open('diff.json') as f:
    diff = json.load(f)
    for _id in diff:
        for week in diff[_id]:
            list_diff.append(str(_id)+'-'+week[5:])
#print(list_diff, len(list_diff))
list_true = list(set(list_ori) - set(list_diff))
#print(list_true, len(list_true))
#print(list_ano_id)
for i in list_ano_id:
    if list_ano_id.count(i) ==1:
        one_time.append(i)
print(one_time, len(one_time))

['"/wzu7i9b!', '0Pk86~O2.', '0&5`o]6FI', 'J1R%u6^h.', '1<YsKp^|4', '@SLw*nK~U5', 'Y/v,H:)\\.', '<@Pm:6pFO+', "?@w,y%ldM'", '"TPk}p$D9\'', "J=&kr)|'7I", "J<xRCG('.4", '1*wNqkn!\\7', 'J-PNH$29b5', 'BsqmQEpdb7', 'fEG:Vp{d#', 'Y0;`7VO|#4', '?/Nq)Kn^p', 'J-&s;z^c', '=GX7nK>b#', '-*[zQC{W#', 'L&Nyp6FU.', ',*jHfV:pt.', '0PCfE)t', 'T@SykXo]gb', 'J*=sr6Kg~', "JYRzhfVD2'", 'R&G}frnip|', '/BLyN)$Mh.', '"1jysk>V$7', '0-PTYqn62', "Smeri{ng'`", "@L&zC`V'I", 'TYLv]p~6O\\', '[mr>:i9\\', '=l;)Ki$#^.', '<LeqGrdO\\`', ',NHX]~^O', 'xH.QGfr)!', 'Z5)6p$FDW_', "[BZmA8ni'7", '"J1wvXV{gI', '?TE{g^t(M+', '0LNk}on{~', '/,f>nK$(5', '/Lvsz]MWU7', '@wQGi6M\\7', '1PLHtng+(4', 'x@&mqX$2!5', 'PxBRqfoD_', '"B&zu}c7_', 'v=&BmZ;63W', '[v=Hk]K{\\#', '0,[Llkr]{', 'P/ju{~^O4', 'J@CE}V{~^g', '?01[sqX)~+', 'xD%]i3|U`', '"1@5>a6$7_', 'zA}r62Uc.I', '0s^O|#_', 'YxLN5g^W#I', "?LZE$t'75", '4PS8i|Wc`', '0-,N~9.', '1-YBvX>{(_', '4T-<Zk6t7I', 'wytzf]nD5', 'PxB%}$+c5', 'vLjNHOuQtU', '0vBjNe8uF#', '?1Gi!^(U.5', '=m.kQg3(\\!', '"T<xZnK!

In [75]:
id_true = list()
with open(file_compare) as o:
    ori_file = json.load(o)
    for id_week in list_ori:
        w = '2015-'+id_week[-2:]
        _id = id_week[:-3]
        anon_id = ori_file[_id][w][0]
        #print(anon_id, type(anon_id))
        if (str(anon_id) in one_time):
            id_true.append(id_week)
print(id_true, len(id_true))
        
    

['1-10', '1-11', '1-12', '1-13', '1-14', '1-15', '1-18', '2-11', '2-14', '2-15', '107-12', '98-10', '98-11', '21-11', '9-10', '9-16', '6-10', '6-11', '6-12', '6-15', '7-15', '7-17', '8-10', '8-11', '8-13', '8-16', '8-17', '11-11', '17-12', '17-13', '15-10', '15-11', '15-12', '44-10', '44-13', '42-11', '42-13', '42-14', '42-15', '16-10', '16-12', '23-11', '32-12', '32-14', '32-15', '32-16', '32-17', '32-18', '24-11', '24-12', '24-15', '24-16', '37-11', '13-10', '13-12', '13-13', '13-16', '13-17', '13-19', '13-20', '26-10', '26-12', '26-13', '26-15', '26-16', '26-17', '26-18', '26-19', '31-11', '31-13', '31-14', '31-15', '28-12', '28-13', '89-18', '29-11', '29-13', '27-11', '27-12', '27-13', '27-14', '27-16', '30-12', '30-13', '30-14', '30-19', '34-12', '38-12', '36-18', '39-10', '39-18', '35-13', '35-15', '54-11', '54-13', '54-14', '41-10', '41-11', '41-12', '41-14', '41-15', '55-11', '55-12', '49-12', '49-13', '49-14', '49-16', '50-12', '50-13', '50-15', '50-17', '50-19', '43-11', '43-

In [81]:
nega_lat = anon.loc[anon['latitude'] < 0].groupby(['id','week'])['latitude'].mean()
print(nega_lat)
print(len(nega_lat))

id    week
1028  13     -12.302976
109   20     -17.451698
134   19      -0.571292
141   11      -5.749388
143   14      -1.644761
211   15      -1.611280
213   14     -12.338260
225   15      -6.273959
245   18     -12.296404
258   20     -12.288348
271   17      -0.570891
289   10      -0.562153
29    17      -1.611321
296   13      -0.768294
324   10      -0.574468
362   17     -12.287612
367   10      -0.569978
4     10      -0.573120
406   10      -0.569984
408   12      -1.067864
420   11      -0.606418
470   15     -12.056009
528   10      -0.570172
567   10     -14.733248
661   15      -1.657086
708   16      -6.269617
731   16      -1.248874
736   12     -12.289691
746   12      -0.691173
      13      -1.124147
755   19      -9.876525
784   11      -0.578573
793   11      -0.575497
823   19      -4.351480
860   13      -0.574864
865   14      -0.996224
871   10      -0.571390
873   18      -1.618034
880   14      -1.612137
883   16     -12.288456
897   20      -4.858576
911  

In [19]:
#df['latitude'] = df['latitude'].round(decimals=2)    
#df['longitude'] = df['longitude'].round(decimals=2)

nega_lat = df.loc[df['latitude'] < 0].groupby(['id','week'])['latitude'].mean()

print(nega_lat)
print(len(nega_lat))

id   week
1    15     -97.749255
     16     -97.688508
     18      -1.798019
     20      -5.451773
4    20     -17.451698
8    11      -0.623015
     13      -0.574869
     16      -1.248818
11   10      -0.562127
     11      -0.575139
13   10      -0.569981
14   10      -0.577909
     11      -0.566633
15   10      -0.569962
16   10      -0.571382
17   10      -0.574534
     11      -0.578577
21   10      -0.583377
     11      -0.566780
23   10      -0.572587
24   14      -1.644390
     15      -1.657268
     17      -0.570890
     19      -0.571296
25   10      -0.573133
31   12      -1.068339
     13      -1.124160
     14      -0.996729
42   10     -46.863902
50   11      -5.749388
51   15      -1.672635
59   14     -32.919764
     18      -3.856955
62   15      -6.273879
     16      -6.269619
73   10      -0.569804
     11      -0.607911
     12      -0.689934
     13      -0.768109
     14      -1.612145
     15      -1.611281
     17      -1.611318
110  10      -1.670039
N

In [425]:
pos_long = anon.loc[(anon['longitude'] >= 0)].groupby(['id','week'])['longitude'].unique()#agg(pd.Series.mode)
print(pos_long)
print(len(pos_long))

id   week
0    10      [45.78, 45.79, 44.84, 44.83, 44.85]
1    16             [45.77, 45.78, 45.79, 45.76]
10   16                    [45.78, 45.77, 45.79]
100  18                           [45.78, 45.77]
101  20                                  [45.78]
                            ...                 
95   16       [45.78, 45.8, 45.77, 45.76, 45.79]
96   19                    [45.53, 45.78, 45.79]
97   11                    [45.78, 45.76, 45.77]
98   10             [45.78, 45.79, 45.77, 45.76]
99   10                                  [45.78]
Name: longitude, Length: 449, dtype: object
449


In [57]:
test_la = anon.loc[(anon['latitude'] >10)&(anon['latitude']<11)].groupby(['week'])['id'].unique()#agg(pd.Series.mode)
print(test_la)
test_lo = anon.loc[(anon['longitude'] >58)&(anon['longitude']<59)].groupby(['week'])['id'].unique()#agg(pd.Series.mode)
print(test_lo)
print(type(test_lo))

Series([], Name: id, dtype: object)
Series([], Name: id, dtype: object)
<class 'pandas.core.series.Series'>


In [429]:
temp = np.sort(list_id_ori)
test = {}
count = 0
for i in temp:
    test[int(i)] = {}
    for w in id_week_ori[i]:
        test[int(i)][int(w)] = [count]
        count +=1

print(test)

[  1   2   4   5   6   7   8   9  11  13  14  15  16  17  18  21  23  24
  25  26  27  28  29  30  31  32  34  35  36  37  38  39  41  42  43  44
  48  49  50  51  52  53  54  55  58  59  60  62  63  65  66  67  68  69
  70  71  72  73  75  77  78  81  83  84  87  89  98 107 110]
{1: {10: [0], 11: [1], 12: [2], 13: [3], 14: [4], 15: [5], 16: [6], 17: [7], 18: [8], 19: [9], 20: [10]}, 2: {10: [11], 11: [12], 12: [13], 13: [14], 14: [15], 15: [16], 16: [17], 17: [18], 18: [19], 19: [20], 20: [21]}, 4: {10: [22], 11: [23], 12: [24], 13: [25], 14: [26], 15: [27], 16: [28], 17: [29], 18: [30], 19: [31], 20: [32]}, 5: {10: [33], 11: [34]}, 6: {10: [35], 11: [36], 12: [37], 13: [38], 14: [39], 15: [40], 16: [41]}, 7: {10: [42], 11: [43], 12: [44], 13: [45], 14: [46], 15: [47], 16: [48], 17: [49]}, 8: {10: [50], 11: [51], 12: [52], 13: [53], 14: [54], 15: [55], 16: [56], 17: [57]}, 9: {10: [58], 11: [59], 12: [60], 13: [61], 14: [62], 15: [63], 16: [64], 19: [65]}, 11: {10: [66], 11: [67]}, 13

In [58]:
t = df.loc[(df.latitude > 17)&(df.latitude <18),['id','week']].drop_duplicates()
print(str(t.id))

25045919    67
Name: id, dtype: int64


In [14]:
pos_long = df.loc[(df['longitude'] >= 0)].groupby(['id','week'])['longitude'].unique()#agg(pd.Series.mode)
print(pos_long)
print(len(pos_long))

id   week
1    10      [45.77, 45.78, 45.79, 45.74, 44.43, 45.82, 45....
     11      [45.36, 45.35, 45.34, 45.33, 45.32, 45.31, 45....
     12      [45.77, 45.78, 45.79, 45.76, 45.75, 45.74, 45....
     13      [43.58, 43.59, 43.57, 43.56, 43.55, 43.54, 43....
     14      [45.77, 45.78, 45.76, 45.75, 45.79, 46.34, 46....
                                   ...                        
107  10      [45.79, 45.78, 45.77, 45.76, 45.75, 45.74, 45....
     11      [45.78, 45.79, 45.77, 45.76, 48.87, 48.86, 48.84]
     12                                         [45.79, 45.78]
110  10      [48.89, 48.84, 48.83, 48.82, 48.81, 48.8, 48.7...
     11                           [48.86, 48.89, 48.83, 48.87]
Name: longitude, Length: 449, dtype: object
449


In [26]:
test = df.loc[(df['latitude'] >10)&(df['latitude']<11)].groupby(['id','week'])['latitude'].count()#agg(pd.Series.mode)
print(test)

test_lo = df.loc[(df['longitude'] >58)&(df['longitude']<59)].groupby(['id','week'])['longitude'].count()#agg(pd.Series.mode)
print(test_lo)

id  week
9   11      1255
Name: latitude, dtype: int64
id  week
83  11      26448
Name: longitude, dtype: int64


In [26]:
data = {}

for i in list_id_ori:
    data[str(i)] = {}
    for w in id_week_ori[i]:
        data[str(i)]['2015'+'-'+str(w)] = [str(i)]
with open('attack_sibyl483', 'w') as fp:
    json.dump(data, fp, indent=1)
